In [ ]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.outcome import Standardize
from botorch.models.transforms.input import Normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.optim import optimize_acqf
from gpytorch.likelihoods import GaussianLikelihood
import gpytorch
from botorch.models.transforms import Normalize
torch.set_default_dtype(torch.float64)

In [1]:
area1 = 0.001485182056299268
area2 = 0.0019705673066026984
area3 = 0.004624429474288485
area4 = 0.0018803643706964024
area5 = 0.00043436256803102525

In [ ]:

train_X = torch.tensor([[0.81411, 0.00011, 0.0004598], [0.88812, 0.00012, 0.0005016], [0.96213, 0.00013, 0.0005434], [0.66609, 0.00009, 0.0003762], [0.59208, 0.00008, 0.0003344]])
train_Y = torch.tensor([[area1], [area2], [area3], [area4], [area5]])  

bounds = torch.tensor([
    [0.1, 0.00005, 0.00020], 
    [1, 0.00014, 0.00065]   
], dtype=torch.float64
)


outcome_transform = Standardize(m=1)

# Optimization loop
num_iterations = 25  
tolerance = 1e-6  
previous_best_area = float('inf')

for i in range(num_iterations):
    # Fit the GP model
    model = SingleTaskGP(train_X, train_Y, input_transform=Normalize(d=3), outcome_transform=outcome_transform)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

   
    acq_func = LogExpectedImprovement(model, best_f=train_Y.min(), maximize=False)

    
    candidate, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=1,  
        num_restarts=10, 
        raw_samples=50,  
    )

    # Print the suggested new parameters
    print(f"Iteration {i+1}: Suggested new parameters (n, tau_star, B):", candidate)

    # Manually input the new area value for the suggested parameters
    new_area = float(input("Enter the new area value for the suggested parameters: "))

    train_X = torch.cat([train_X, candidate], dim=0)
    train_Y = torch.cat([train_Y, torch.tensor([[new_area]])], dim=0)

  
    current_best_area = train_Y.min().item()
    print(f"Iteration {i+1}: Best area so far: {current_best_area}")

    if abs(previous_best_area - current_best_area) < tolerance:
        print(f"Converged after {i+1} iterations with minimal error.")
        break

    previous_best_area = current_best_area

# Final output of the best found parameters and corresponding area
best_params = train_X[train_Y.argmin()]
best_area = train_Y.min().item()
print(f"Best parameters (n, tau_star, B): {best_params}, with area: {best_area}")